In [ ]:
import pandas as pd
import numpy as np
from wbe.constants import DATA_PATH

In [ ]:
data = pd.read_csv(DATA_PATH / "wbe/cdc_data.csv", parse_dates=True, index_col="sample_collect_date")

In [ ]:
data["linear_pcr_conc"] = np.nan
log_mask = data["pcr_target_units"] == "log10 copies/l wastewater"
linear_mask = ~log_mask
data.loc[log_mask, "linear_pcr_conc"] = 10.0 ** data.loc[log_mask, "pcr_target_avg_conc"]
data.loc[linear_mask, "linear_pcr_conc"] = data.loc[linear_mask, "pcr_target_avg_conc"]

In [ ]:
solid_mask = data["pcr_target_units"] == "copies/g dry sludge"
liquid_mask = ~solid_mask
solid_data = data[solid_mask].copy()
liquid_data = data[liquid_mask].copy()